In [1]:
import numpy as np
import pandas as pd
from numpy import array
import matplotlib.pyplot as plt

from sklearn.feature_selection import f_classif, mutual_info_classif, SelectFdr

from sklearn.feature_selection import GenericUnivariateSelect
from sklearn.svm import LinearSVC, NuSVC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier  

import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

from warnings import filterwarnings
filterwarnings('ignore')


In [2]:
class hyper_model:
    
    def __init__(self,model_name,X_train,y_train,X_test,y_test):
        
        self.model_name = model_name
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.best_model = None
        self.best_params = None
        self.best_model_perf = None
        self.all_models_perf = pd.DataFrame(None,columns=['Model Iteration','Score'])
        self.stock_model = None
        self.rd_model = None
        self.grid_model = None


    def fit(self):
        
 
        
        # Linear SVM



        if self.model_name == 'linear_svm':

            # Initial Model Fit with stock hyper_params    

            model = LinearSVC(random_state=0, tol=1e-5)

            model.fit(self.X_train,self.y_train)

            init_score = model.score(self.X_test,self.y_test)

            self.all_models_perf = self.all_models_perf.append({'Model Iteration': 'Stock Params',
                                        'Score':init_score}, ignore_index=True)

            print('Initial Model with Stock Params Score : {}'.format(init_score))

            # Random Search Cross Validation

            params = {'C': [0.1, 1, 10, 100, 1000]}  



            search_linear_svm = RandomizedSearchCV(model,
                                                   param_distributions=params,
                                                   random_state=42, 
                                                   n_iter=300, 
                                                   cv=4, 
                                                   verbose=1, 
                                                   n_jobs=1, 
                                                   return_train_score=True)

            search_linear_svm.fit(X,y)

            best_rd_linear_svm = search_linear_svm.best_estimator_

            best_rd_linear_svm_score = best_rd_linear_svm.score(X_test, y_test)

            self.all_models_perf = self.all_models_perf.append({'Model Iteration': 'Random HP Search Best Model',
                                'Score':best_rd_linear_svm_score }, ignore_index=True)

            rd_score_improv = ((best_rd_linear_svm_score - init_score)/init_score) * 100

            print('Random HP Search Best Model Score : {}'.format(best_rd_linear_svm_score))

            print('Improvement of {}% relative to the initial model'.format(rd_score_improv))

            # Further Grid Search Cross Validation

            params = {'C': np.linspace(search_linear_svm.best_params_['C']-5,
                                       search_linear_svm.best_params_['C']+5,
                                       10)}  

            grid_search_linear_svm = GridSearchCV(model,
                                             param_grid=params, 
                                             cv=4, 
                                             verbose=1, 
                                             n_jobs=1)

            grid_search_linear_svm.fit(X,y)

            grid_best_rd_linear_svm = grid_search_linear_svm.best_estimator_

            grid_best_rd_linear_svm_score = grid_best_rd_linear_svm.score(X_test, y_test)

            self.all_models_perf = self.all_models_perf.append({'Model Iteration': 'Further Grid HP Search Best Model',
                                'Score':grid_best_rd_linear_svm_score}, ignore_index=True)

            grid_score_improv = ((grid_best_rd_linear_svm_score - init_score)/init_score) * 100

            print('Further Grid HP Search Best Model Score : {}'.format(grid_best_rd_linear_svm_score))

            print('Improvement of {}% relative to the initial model'.format(grid_score_improv))


            mask_best = self.all_models_perf['Score'] == self.all_models_perf['Score'].max()

            overall_best = str(self.all_models_perf['Model Iteration'][self.all_models_perf['Score'][mask_best].index])

            print('Overall Best Model : {}'.format(overall_best))

            if overall_best == 'Stock Params':

                self.best_model = model
                self.best_params = model.get_params()

            elif overall_best == 'Random HP Search Best Model':

                self.best_model = best_rd_linear_svm
                self.best_params = best_rd_linear_svm.get_params()

            elif overall_best == 'Further Grid HP Search Best Model':

                self.best_model = grid_best_rd_linear_svm
                self.best_params = grid_best_rd_linear_svm.get_params()


            self.stock_model = model
            self.rd_model = best_rd_linear_svm
            self.grid_model = grid_best_rd_linear_svm



        # Non Linear SVM


        elif self.model_name == 'non_linear_svm':

            # Initial Model Fit with stock hyper_params

            model = NuSVC(gamma='auto',random_state=True,kernel='poly')

            model.fit(self.X_train,self.y_train)

            init_score = model.score(self.X_test,self.y_test)

            self.all_models_perf = self.all_models_perf.append({'Model Iteration': 'Stock Params',
                                        'Score':init_score}, ignore_index=True)

            print('Initial Model with Stock Params Score : {}'.format(init_score))

            # Random Search Cross Validation


            params = {'nu': [0.01, 0.3, 0.5, 0.7, 1],  
                          'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
                          'kernel': ['rbf','poly','sigmoid']
                         }  

            search_non_linear_svm = RandomizedSearchCV(model,
                                                   param_distributions=params,
                                                   random_state=42, 
                                                   n_iter=300, 
                                                   cv=4, 
                                                   verbose=1, 
                                                   n_jobs=1, 
                                                   return_train_score=True)

            search_non_linear_svm.fit(X,y)

            best_rd_non_linear_svm = search_non_linear_svm.best_estimator_

            best_rd_non_linear_svm_score = best_rd_non_linear_svm.score(X_test, y_test)

            self.all_models_perf = self.all_models_perf.append({'Model Iteration': 'Random HP Search Best Model',
                                'Score':best_rd_non_linear_svm_score}, ignore_index=True)

            rd_score_improv = ((best_rd_non_linear_svm_score - init_score)/init_score) * 100

            print('Random HP Search Best Model Score : {}'.format(best_rd_non_linear_svm_score))

            print('Improvement of {}% relative to the initial model'.format(rd_score_improv))
            
            

            # Further Grid Search Cross Validation

            params = {'nu': np.linspace(search_non_linear_svm.best_params_['nu']-0.1,
                                       search_non_linear_svm.best_params_['nu']+0.1,
                                       10),  
                      'gamma': np.linspace(search_non_linear_svm.best_params_['gamma']-0.1,
                                       search_non_linear_svm.best_params_['gamma']+0.1,
                                       10), 
                      'kernel': [search_non_linear_svm.best_params_['kernel']]
                     }  

            grid_search_non_linear_svm = GridSearchCV(model,
                                             param_grid=params, 
                                             cv=4, 
                                             verbose=1, 
                                             n_jobs=1)

            grid_search_non_linear_svm.fit(X,y)

            grid_best_rd_non_linear_svm = grid_search_non_linear_svm.best_estimator_

            grid_best_rd_non_linear_svm_score = grid_best_rd_non_linear_svm.score(X_test, y_test)

            self.all_models_perf = self.all_models_perf.append({'Model Iteration': 'Further Grid HP Search Best Model',
                                'Score':grid_best_rd_non_linear_svm_score}, ignore_index=True)

            grid_score_improv = ((grid_best_rd_non_linear_svm_score - init_score)/init_score) * 100

            print('Further Grid HP Search Best Model Score : {}'.format(grid_best_rd_non_linear_svm_score))

            print('Improvement of {}% relative to the initial model'.format(grid_score_improv))

            mask_best = self.all_models_perf['Score'] == self.all_models_perf['Score'].max()

            overall_best = str(self.all_models_perf['Model Iteration'][self.all_models_perf['Score'][mask_best].index])

            print('Overall Best Model : {}'.format(overall_best))

            if overall_best == 'Stock Params':

                self.best_model = model
                self.best_params = model.get_params()

            elif overall_best == 'Random HP Search Best Model':

                self.best_model = best_rd_non_linear_svm
                self.best_params = best_rd_non_linear_svm.get_params()

            elif overall_best == 'Further Grid HP Search Best Model':

                self.best_model = grid_best_rd_non_linear_svm
                self.best_params = grid_best_rd_non_linear_svm.get_params()

            self.stock_model = model
            self.rd_model = best_rd_non_linear_svm
            self.grid_model = grid_best_rd_non_linear_svm


        # Random Forest Classifier


        elif self.model_name == 'random_forest_classifier':


            # Initial Model Fit with stock hyper_params

            model = RandomForestClassifier(verbose=1)

            model.fit(self.X_train,self.y_train)

            init_score = model.score(self.X_test,self.y_test)

            self.all_models_perf = self.all_models_perf.append({'Model Iteration': 'Stock Params',
                                        'Score':init_score}, ignore_index=True)

            print('Initial Model with Stock Params Score : {}'.format(init_score))

            # Random Search Cross Validation


            params = {'bootstrap': [True, False],
                      'max_depth': [50, 60, 70, 80, 90, 100],
                      'max_features': ['auto', 'sqrt'],
                      'min_samples_leaf': [1, 2, 4],
                      'min_samples_split': [2, 5, 10],
                      'n_estimators': [1000, 1200, 1400, 1600, 1800, 2000]
                     }  

            search_rf = RandomizedSearchCV(model,
                                                   param_distributions=params,
                                                   random_state=42, 
                                                   n_iter=30, 
                                                   cv=4, 
                                                   verbose=1, 
                                                   n_jobs=1, 
                                                   return_train_score=True)

            search_rf.fit(X,y)

            best_rd_rf = search_rf.best_estimator_

            best_rd_rf_score = best_rd_rf.score(X_test, y_test)

            self.all_models_perf = self.all_models_perf.append({'Model Iteration': 'Random HP Search Best Model',
                                'Score':best_rd_rf_score}, ignore_index=True)

            rd_score_improv = ((best_rd_rf_score - init_score)/init_score) * 100

            print('Random HP Search Best Model Score : {}'.format(best_rd_rf_score))

            print('Improvement of {}% relative to the initial model'.format(rd_score_improv))


            # Further Grid Search Cross Validation

            params = {'bootstrap': [search_rf.best_params_['bootstrap']],
                      'max_depth': np.linspace(search_rf.best_params_['max_depth']-15,
                                       search_rf.best_params_['max_depth']+15,
                                       10),
                      'max_features': [search_rf.best_params_['max_features']],
                      'min_samples_leaf': np.linspace(search_rf.best_params_['min_samples_leaf']-1,
                                       search_rf.best_params_['min_samples_leaf']+1,
                                       10),
                      'min_samples_split': np.linspace(search_rf.best_params_['min_samples_split']-1,
                                       search_rf.best_params_['min_samples_split']+1,
                                       10),
                      'n_estimators': [search_rf.best_params_['n_estimators']]
                     }  

            grid_search_rf = GridSearchCV(model,
                                             param_grid=params, 
                                             cv=4, 
                                             verbose=1, 
                                             n_jobs=1)

            grid_search_rf.fit(X,y)

            grid_best_rd_rf = grid_search_rf.best_estimator_

            grid_best_rd_rf_score = grid_best_rd_rf.score(X_test, y_test)

            self.all_models_perf = self.all_models_perf.append({'Model Iteration': 'Further Grid HP Search Best Model',
                                'Score':grid_best_rd_rf_score}, ignore_index=True)

            grid_rd_score_improv = ((grid_best_rd_rf_score - init_score)/init_score) * 100

            print('Further Grid HP Search Best Model Score : {}'.format(grid_best_rd_rf_score))

            print('Improvement of {}% relative to the initial model'.format(grid_rd_score_improv))

            mask_best = self.all_models_perf['Score'] == self.all_models_perf['Score'].max()

            overall_best = str(self.all_models_perf['Model Iteration'][self.all_models_perf['Score'][mask_best].index])

            print('Overall Best Model : {}'.format(overall_best))

            if overall_best == 'Stock Params':

                self.best_model = model
                self.best_params = model.get_params()

            elif overall_best == 'Random HP Search Best Model':

                self.best_model = best_rd_rf
                self.best_params = best_rd_rf.get_params()

            elif overall_best == 'Further Grid HP Search Best Model':

                self.best_model = grid_best_rd_rf
                self.best_params = grid_best_rd_rf.get_params()


            self.stock_model = model
            self.rd_model = best_rd_rf
            self.grid_model = grid_best_rd_rf




        # XG Boost Classifier


        elif self.model == 'xg_boost':


            # Initial Model Fit with stock hyper_params

            model = xgb.XGBClassifier()

            model.fit(self.X_train,self.y_train)

            init_score = model.score(self.X_test,self.y_test)

            self.all_models_perf = self.all_models_perf.append({'Model Iteration': 'Stock Params',
                                        'Score':init_score}, ignore_index=True)

            print('Initial Model with Stock Params Score : {}'.format(init_score))




            # Random Search Cross Validation


            params = {
                "colsample_bytree": uniform(0.7, 0.3),
                "gamma": uniform(0, 0.5),
                "learning_rate": uniform(0.03, 0.3), 
                "max_depth": randint(2, 6), 
                "n_estimators": randint(100, 150),
                "subsample": uniform(0.6, 0.4)
                }  

            search_xgb = RandomizedSearchCV(model,
                                                   param_distributions=params,
                                                   random_state=42, 
                                                   n_iter=300, 
                                                   cv=4, 
                                                   verbose=1, 
                                                   n_jobs=1, 
                                                   return_train_score=True)

            search_xgb.fit(X,y)

            best_rd_xgb = search_xgb.best_estimator_

            best_rd_xgb_score = best_rd_xgb.score(X_test, y_test)

            self.all_models_perf = self.all_models_perf.append({'Model Iteration': 'Random HP Search Best Model',
                                'Score':best_rd_xgb_score}, ignore_index=True)

            rd_score_improv = ((best_rd_xgb_score - init_score)/init_score) * 100

            print('Random HP Search Best Model Score : {}'.format(best_rd_xgb_score))

            print('Improvement of {}% relative to the initial model'.format(rd_score_improv))


            # Further Grid Search Cross Validation



            params = {
                "colsample_bytree": np.linspace(search_xgb.best_params_['colsample_bytree']-0.1,
                                       search_xgb.best_params_['colsample_bytree']+0.1,
                                       10),
                "gamma": np.linspace(search_xgb.best_params_['gamma']-0.1,
                                       search_xgb.best_params_['gamma']+0.1,
                                       10),
                "learning_rate": np.linspace(search_xgb.best_params_['learning_rate']-0.1,
                                       search_xgb.best_params_['learning_rate']+0.1,
                                       10), 
                "max_depth": [search_xgb.best_params_['max_depth']], 
                "n_estimators": [search_xgb.best_params_['n_estimators']],
                "subsample": np.linspace(search_xgb.best_params_['subsample']-0.1,
                                       search_xgb.best_params_['subsample']+0.1,
                                       10)
                }  

            grid_search_xgb = GridSearchCV(model,
                                             param_grid=params, 
                                             cv=4, 
                                             verbose=1, 
                                             n_jobs=1)

            grid_search_xgb.fit(X,y)

            grid_best_rd_xgb = grid_search_xgb.best_estimator_

            grid_best_rd_xgb_score = grid_best_rd_xgb.score(X_test, y_test)

            self.all_models_perf = self.all_models_perf.append({'Model Iteration': 'Further Grid HP Search Best Model',
                                'Score':grid_best_rd_xgb_score}, ignore_index=True)

            grid_rd_score_improv = ((grid_best_rd_xgb_score - init_score)/init_score) * 100

            print('Further Grid HP Search Best Model Score : {}'.format(grid_best_rd_xgb_score))

            print('Improvement of {}% relative to the initial model'.format(grid_rd_score_improv))

            mask_best = self.all_models_perf['Score'] == self.all_models_perf['Score'].max()

            overall_best = str(self.all_models_perf['Model Iteration'][self.all_models_perf['Score'][mask_best].index])

            print('Overall Best Model : {}'.format(overall_best))

            if overall_best == 'Stock Params':

                self.best_model = model
                self.best_params = model.get_params()

            elif overall_best == 'Random HP Search Best Model':

                self.best_model = best_rd_xgb
                self.best_params = best_rd_xgb.get_params()

            elif overall_best == 'Further Grid HP Search Best Model':

                self.best_model = grid_best_rd_xgb
                self.best_params = grid_best_rd_xgb.get_params()

            self.stock_model = model
            self.rd_model = best_rd_xgb
            self.grid_model = grid_best_rd_xgb
                
                

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd drive/MyDrive/CEA/

/content/drive/MyDrive/CEA


#### Data Preparation

In [5]:
X = np.load('X_pref_lss_shift.npy')

In [6]:
X.shape = (480, 11457)

In [7]:
y = np.load('y_pref_lss_shift.npy')

In [8]:
y.shape = (480, 1)

In [22]:
df.columns[:-1]

Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,
       ...
       11447, 11448, 11449, 11450, 11451, 11452, 11453, 11454, 11455, 11456],
      dtype='object', length=11457)

In [27]:
df = pd.DataFrame(X,columns=range(11457))
df['Pleasance'] = y
df = df[[df.columns[-1]] + list(df.columns[:-1])]

In [28]:
df.head()

Pleasance         0         1  ...     11454     11455     11456
0        HP -1.704227  1.925462  ...  0.700836  0.093239  0.620513
1        HP  0.784934 -0.431708  ... -2.193575  0.633638 -1.915317
2        HP  1.651002 -0.713096  ...  0.023924 -2.355849 -1.677730
3        HP -2.252201  0.746380  ... -1.696722  2.214205  1.023099
4        HP  0.282161 -0.717110  ...  1.476642  0.584545  1.143373

[5 rows x 11458 columns]

## Features Selection

### Filter

In [9]:
transformer = GenericUnivariateSelect(mutual_info_classif, mode='k_best', param=3)

In [10]:
X_trans = transformer.fit_transform(X, y)

In [11]:
X_trans.shape

(480, 3)

#### PCA

In [12]:
from sklearn.decomposition import PCA

Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,
       ...
       11447, 11448, 11449, 11450, 11451, 11452, 11453, 11454, 11455, 11456],
      dtype='object', length=11457)

In [30]:
pca = PCA(n_components = 3)

X_pca = pca.fit_transform(df[df.columns[1:]])

In [31]:
tot_var = pca.explained_variance_ratio_.sum()*100

### Data Visualization

In [34]:
import plotly.express as px

##### PCA

In [32]:
X_pca

array([[-52.19988263,   0.06656338,  43.54160776],
       [ 35.92005037, -13.91231222,  -9.53909839],
       [ -1.41984768, -42.20226351,  24.8118998 ],
       ...,
       [  3.91759733,  55.81398304,  19.09824146],
       [-30.45244534, -21.91738147,  24.16519087],
       [ -1.40201966,  32.50277446,  -8.88577992]])

In [36]:
fig = px.scatter_3d(
    X_pca, x=0, y=1, z=2, color=df['Pleasance'],
    title=f'Total Explained Variance: {tot_var:.2f}%',
    labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
)
fig.show()

In [40]:
fig = px.scatter(
    X_pca, x=0, y=1, color=df['Pleasance'],
    labels={'0': 'PC 1', '1': 'PC 2'}
)
fig.show()

##### Features Selection

In [37]:
fig = px.scatter_3d(
    X_trans, x=0, y=1, z=2, color=df['Pleasance'],
    
    labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
)
fig.show()

In [38]:
fig = px.scatter(
    X_trans, x=0, y=1, color=df['Pleasance'],
    
    labels={'0': 'PC 1', '1': 'PC 2'}
)
fig.show()